# Prédictions pour le modele UNET 

In [20]:
# Libraries
import os
import sys
import numpy as np
import skimage.io as io
import skimage.transform as trans
import matplotlib.image as mpimg
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, ZeroPadding2D, Convolution2D, Dropout
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.metrics import f1_score

In [2]:
# Load the model
json_file = open('Model_UNET.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

In [28]:
def data_augmentation(directory_name, seed): 
    datagen = ImageDataGenerator()
    filenames = os.listdir(directory_name)
    
    #create 24 rotated images for one image
    angls = np.arange(0)
    zooms = np.array([1.])
    imgs = []
    
    for i, fileNb in enumerate(filenames):
        full_name = directory_name+fileNb
        img = mpimg.imread(full_name)
        imgr = img_to_array(img)
        for j, angle in enumerate(angls):
            zoom = zooms[j]
            img2 = datagen.apply_transform(x=imgr, transform_parameters={'theta':angle, 'zx':zoom, 'zy':zoom})
            imgs.append(img2)
        sys.stdout.write("\rImage {}/{} is being loaded".format(i+1,len(filenames)))
        sys.stdout.flush()
        
    print(' ... Shuffle data ...')
    imgs1 = np.asarray(imgs)
    #np.random.seed(seed)
    #rand = np.random.randint(imgs1.shape[0], size=imgs1.shape[0])
    #imgs2 = imgs1[rand, :, :, :]
    
    # shows images
    #IMG = array_to_img(imgs2[0,:,:,:])
    #imgplot = plt.imshow(IMG)
    #plt.show()
    return 1

imgs = data_augmentation("../Data/training/images/", 1)

Image 100/100 is being loaded

In [29]:
print(imgs.shape)

(0,)


In [15]:
# Load the images to be predicted (here the testing set)
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

path = "../Data/training/images/"

files = os.listdir(path)
n = 50
imgs = [load_image(path + files[i]) for i in range(n)]

In [16]:
T = ImageDataGenerator()
test_generator  = T.flow(imgs, batch_size=32)

ValueError: ('Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape', (400, 400, 3))

In [8]:
res = model.predict(T, verbose=1)

ValueError: Error when checking input: expected input_1 to have 4 dimensions, but got array with shape (400, 400, 3)

In [15]:
def testGenerator(test_path,num_image = 30,target_size = (256,256),flag_multi_class = False,as_gray = False):
    for i in range(1, num_image+1):
        img = io.imread(os.path.join(test_path,"satImage_%03d.png"%i),as_gray = as_gray)
        
        img = img / 255
        
        img = trans.resize(img,target_size)
        
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        print(img.shape)
        img = np.reshape(img,(1,)+img.shape)
        print(img.shape)
        yield img

In [16]:
test = testGenerator("../Data/training/images/")
#results = model.predict_generator(test,16,verbose=1)#

In [10]:
test

<generator object testGenerator at 0x7fd7081b7048>